#  EPA-122A *Spatial* Data Science


## Lab 2 - part 1: Data Engineering

**TU Delft**<br>
**Q2 2024**<br>
**Instructors:** Giacomo Marangoni, Theodoros Chatzivasileiadis <br>

---


## Table of Contents

* [Learning Goals](#chapter0)
* [Creating data](#chapter1)
    * [DataFrame](#section1_1)
    * [Series](#section1_2)
* [Reading data files](#chapter2)
* [Indexing data](#chapter3)
    * [Native accessors](#section3_1)
    * [Indexing in pandas](#section3_2)
        * [Index-based selection](#section3_2_1)
        * [Label-based selection](#section3_2_2)
        * [Choosing between loc and iloc](#section3_2_3)
        * [Manipulating the index](#section3_2_4)
    * [Conditional selection](#section3_3)
    * [Using indexing to assign data](#section3_4)
* [Groupwise analysis (Advanced skill)](#chapter4)
    * [Multi-indexes](#section4_1)
    * [Sorting](#section4_2)
* [Additional tools](#chapter5)
    * [Summary functions](#section5_1)
    * [Maps (Advanced skill)](#section5_2)
    * [Dtypes](#section5_3)
    * [Missing data](#section5_4)
    * [Renaming](#section5_5)
    * [Combining data](#section5_6)

## Learning Goals <a class="anchor" id="chapter0"></a>

- Understand different core objects, learn how to create data and reading data files.
- Learn the concepts and tools of 'indexing' data.
- Develop an understanding of 'groupwise analysis' and learn the necessary tools.
- Learn additional tools of analysing data as mentioned in TOC.

This notebook covers the basic tools of data analysis that are **needed** for the course

In [ ]:
import pandas as pd

## Creating data  <a class="anchor" id="chapter1"></a>

There are two core objects in pandas: the DataFrame and the Series.


### DataFrame <a class="anchor" id="section1_1"></a>

A DataFrame is a table. It contains an array of individual entries, each of which has a certain value. Each entry corresponds to a row (or record) and a column.

For example, consider the following simple DataFrame:

In [ ]:
pd.DataFrame({"Yes": [50, 21], "No": [131, 2]})


In this example, the "0, No" entry has the value of 131. The "0, Yes" entry has a value of 50, and so on.

DataFrame entries are not limited to integers. For instance, here's a DataFrame whose values are strings:


In [ ]:
pd.DataFrame(
    {"Bob": ["I liked it.", "It was awful."], "Sue": ["Pretty good.", "Bland."]}
)



We are using the `pd.DataFrame()` constructor to generate these DataFrame objects. The syntax for declaring a new one is a dictionary whose keys are the column names (`Bob` and `Sue` in this example), and whose values are a list of entries. This is the standard way of constructing a new DataFrame, and the one you are most likely to encounter.

The dictionary-list constructor assigns values to the column labels, but just uses an ascending count from 0 (0, 1, 2, 3, ...) for the row labels. Sometimes this is OK, but oftentimes we will want to assign these labels ourselves.

The list of row labels used in a DataFrame is known as an Index. We can assign values to it by using an `index` parameter in our constructor:


In [ ]:
pd.DataFrame(
    {"Bob": ["I liked it.", "It was awful."], "Sue": ["Pretty good.", "Bland."]},
    index=["Product A", "Product B"],
)

### Series  <a class="anchor" id="section1_2"></a>

A Series, by contrast, is a sequence of data values. If a DataFrame is a table, a Series is a list. And in fact you can create one with nothing more than a list:


In [ ]:
pd.Series([1, 2, 3, 4, 5])

A Series is, in essence, a single column of a DataFrame. So you can assign column values to the Series the same way as before, using an index parameter. However, a Series does not have a column name, it only has one overall name:

In [ ]:
pd.Series(
    [30, 35, 40], index=["2015 Sales", "2016 Sales", "2017 Sales"], name="Product A"
)



The Series and the DataFrame are intimately related. It's helpful to think of a DataFrame as actually being just a bunch of Series "glued together". We'll see more of this in the next section of this tutorial.


## Reading data files  <a class="anchor" id="chapter2"></a>

Being able to create a DataFrame or Series by hand is handy. But, most of the time, we won't actually be creating our own data by hand. Instead, we'll be working with data that already exists.

Data can be stored in any of a number of different forms and formats. By far the most basic of these is the humble CSV file. When you open a CSV file you get something that looks like this:

`Product A,Product B,Product C,
30,21,9,
35,34,1,
41,11,11`

So a CSV file is a table of values separated by commas. Hence the name: "Comma-Separated Values", or CSV.

Let's now set aside our toy datasets and see what a real dataset looks like when we read it into a DataFrame. We'll use the pd.read_csv() function to read the data into a DataFrame. This goes thusly:

In [ ]:
wine_reviews = pd.read_csv("./data/winemag-data-130k-v2.csv")

We can use the `shape` attribute to check how large the resulting DataFrame is:

In [ ]:
wine_reviews.shape

So our new DataFrame has 130,000 records split across 14 different columns. That's almost 2 million entries!

We can examine the contents of the resultant DataFrame using the head() command, which grabs the first five rows:

In [ ]:
wine_reviews.head()

The `pd.read_csv()` function is well-endowed, with over 30 optional parameters you can specify. For example, you can see in this dataset that the CSV file has a built-in index, which pandas did not pick up on automatically. To make pandas use that column for the index (instead of creating a new one from scratch), we can specify an `index_col`.


In [ ]:
wine_reviews = pd.read_csv("./data/winemag-data-130k-v2.csv", index_col=0)
wine_reviews.head()

## Indexing data   <a class="anchor" id="chapter3"></a>

### Native accessors   <a class="anchor" id="section3_1"></a>

Native Python objects provide good ways of indexing data. Pandas carries all of these over, which helps make it easy to start with.

Consider this DataFrame:

In [ ]:
reviews = pd.read_csv("./data/winemag-data-130k-v2.csv", index_col=0)

In Python, we can access the property of an object by accessing it as an attribute. A `book` object, for example, might have a title property, which we can access by calling `book.title`. Columns in a pandas DataFrame work in much the same way.

Hence to access the `country` property of `reviews` we can use:


In [ ]:
reviews.country  #'country' being the name of the column of interest

If we have a Python dictionary, we can access its values using the indexing ([]) operator. We can do the same with columns in a DataFrame:

In [ ]:
reviews["country"]

These are the two ways of selecting a specific Series out of a DataFrame. Neither of them is more or less syntactically valid than the other, but the indexing operator `[]` does have the advantage that it can handle column names with reserved characters in them (e.g. if we had a `country providence` column, `reviews.country` providence wouldn't work).

Doesn't a pandas Series look kind of like a fancy dictionary? It pretty much is, so it's no surprise that, to drill down to a single specific value, we need only use the indexing operator `[]` once more:


In [ ]:
reviews["country"][0]

### Indexing in pandas   <a class="anchor" id="section3_2"></a>

The indexing operator and attribute selection are nice because they work just like they do in the rest of the Python ecosystem. As a novice, this makes them easy to pick up and use. However, pandas has its own accessor operators, `loc` and `iloc`. For more advanced operations, these are the ones you're supposed to be using.


#### Index-based selection   <a class="anchor" id="section3_2_1"></a>

Pandas indexing works in one of two paradigms. The first is **index-based selection**: selecting data based on its numerical position in the data. `iloc` follows this paradigm.

To select the first row of data in a DataFrame, we may use the following:

In [ ]:
reviews.iloc[0]

Both `loc` and `iloc` are row-first, column-second. This is the opposite of what we do in native Python, which is column-first, row-second.

This means that it's marginally easier to retrieve rows, and marginally harder to get retrieve columns. To get a column with `iloc`, we can do the following:


In [ ]:
reviews.iloc[:, 0]

On its own, the `:` operator, which also comes from native Python, means "everything". When combined with other selectors, however, it can be used to indicate a range of values. For example, to select the country column from just the first, second, and third row, we would do:

In [ ]:
reviews.iloc[:3, 0]



Or, to select just the second and third entries, we would do:


In [ ]:
reviews.iloc[1:3, 0]



It's also possible to pass a list:


In [ ]:
reviews.iloc[[0, 1, 2], 0]

Finally, it's worth knowing that negative numbers can be used in selection. This will start counting forwards from the end of the values. So for example here are the last five elements of the dataset.


In [ ]:
reviews.iloc[-5:]


#### Label-based selection  <a class="anchor" id="section3_2_2"></a>

The second paradigm for attribute selection is the one followed by the `loc` operator: **label-based selection**. In this paradigm, it's the data index value, not its position, which matters.

For example, to get the first entry in `reviews`, we would now do the following:


In [ ]:
reviews.loc[0, "country"]

`iloc` is conceptually simpler than `loc` because it ignores the dataset's indices. When we use `iloc` we treat the dataset like a big matrix (a list of lists), one that we have to index into by position. loc, by contrast, uses the information in the indices to do its work. Since your dataset usually has meaningful indices, it's usually easier to do things using `loc` instead. For example, here's one operation that's much easier using `loc`:


In [ ]:
reviews.loc[:, ["taster_name", "taster_twitter_handle", "points"]]


#### Choosing between `loc` and `iloc`  <a class="anchor" id="section3_2_3"></a>

When choosing or transitioning between `loc` and `iloc`, there is one "gotcha" worth keeping in mind, which is that the two methods use slightly different indexing schemes.

`iloc` uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded. So `0:10` will select entries `0,...,9`. `loc`, meanwhile, indexes inclusively. So `0:10` will select entries `0,...,10`.

Why the change? Remember that loc can index any stdlib type: strings, for example. If we have a DataFrame with index values `Apples, ..., Potatoes, ...,` and we want to select "all the alphabetical fruit choices between Apples and Potatoes", then it's a lot more convenient to index `df.loc['Apples':'Potatoes']` than it is to index something like `df.loc['Apples', 'Potatoet]` (`t` coming after `s` in the alphabet).

This is particularly confusing when the DataFrame index is a simple numerical list, e.g. `0,...,1000`. In this case `df.iloc[0:1000]` will return 1000 entries, while `df.loc[0:1000]` return 1001 of them! To get 1000 elements using `loc`, you will need to go one lower and ask for `df.loc[0:999]`.

Otherwise, the semantics of using `loc` are the same as those for `iloc`.


#### Manipulating the index  <a class="anchor" id="section3_2_4"></a>

Label-based selection derives its power from the labels in the index. Critically, the index we use is not immutable. We can manipulate the index in any way we see fit.

The `set_index()` method can be used to do the job. Here is what happens when we `set_index` to the `title` field:

In [ ]:
reviews.set_index("title")

This is useful if you can come up with an index for the dataset which is better than the current one.


### Conditional selection  <a class="anchor" id="section3_3"></a>

So far we've been indexing various strides of data, using structural properties of the DataFrame itself. To do interesting things with the data, however, we often need to ask questions based on conditions.

For example, suppose that we're interested specifically in better-than-average wines produced in Italy.

We can start by checking if each wine is Italian or not:


In [ ]:
reviews.country == "Italy"

This operation produced a Series of `True/False` booleans based on the `country` of each record. This result can then be used inside of `loc` to select the relevant data:

In [ ]:
reviews.loc[reviews.country == "Italy"]

This DataFrame has ~20,000 rows. The original had ~130,000. That means that around 15% of wines originate from Italy.

We also wanted to know which ones are better than average. Wines are reviewed on a 80-to-100 point scale, so this could mean wines that accrued at least 90 points.

We can use the ampersand (`&`) to bring the two questions together

In [ ]:
reviews.loc[(reviews.country == "Italy") & (reviews.points >= 90)]

Suppose we'll buy any wine that's made in Italy or which is rated above average. For this we use a pipe (`|`):

In [ ]:
reviews.loc[(reviews.country == "Italy") | (reviews.points >= 90)]



Pandas comes with a few built-in conditional selectors, two of which we will highlight here.

The first is `isin`. `isin` is lets you select data whose value "is in" a list of values. For example, here's how we can use it to select wines only from Italy or France:


In [ ]:
reviews.loc[reviews.country.isin(["Italy", "France"])]

The second is `isnull` (and its companion `notnull`). These methods let you highlight values which are (or are not) empty (`NaN`). For example, to filter out wines lacking a price tag in the dataset, here's what we would do:

In [ ]:
reviews.loc[reviews.price.notnull()]


### Use indexing to assign data   <a class="anchor" id="section3_4"></a>

Going the other way, assigning data to a DataFrame is easy. You can assign either a constant value:


In [ ]:
reviews["critic"] = "everyone"
reviews["critic"]

Or with an iterable of values:

In [ ]:
reviews["index_backwards"] = range(len(reviews), 0, -1)
reviews["index_backwards"]

## Groupwise analysis (Advanced skill)  <a class="anchor" id="chapter4"></a>

One function we've been using heavily thus far is the `value_counts()` function. We can replicate what `value_counts()` does by doing the following:

In [ ]:
reviews.groupby("points").points.count()

`groupby()` created a group of reviews which allotted the same point values to the given wines. Then, for each of these groups, we grabbed the `points()` column and counted how many times it appeared. `value_counts()` is just a shortcut to this `groupby()` operation.

We can use any of the summary functions we've used before with this data. For example, to get the cheapest wine in each point value category, we can do the following:


In [ ]:
reviews.groupby("points").price.min()

You can think of each group we generate as being a slice of our DataFrame containing only data with values that match. This DataFrame is accessible to us directly using the `apply()` method, and we can then manipulate the data in any way we see fit. For example, here's one way of selecting the name of the first wine reviewed from each winery in the dataset:


In [ ]:
reviews.groupby("winery").apply(lambda df: df.title.iloc[0], include_groups=False)

For even more fine-grained control, you can also group by more than one column. For an example, here's how we would pick out the best wine by country and province:


In [ ]:
reviews.groupby(["country", "province"]).apply(lambda df: df.loc[df.points.idxmax()], include_groups=False)

Another `groupby()` method worth mentioning is `agg()`, which lets you run a bunch of different functions on your DataFrame simultaneously. For example, we can generate a simple statistical summary of the dataset as follows:


In [ ]:
reviews.groupby(["country"]).price.agg([len, "min", "max"])

Effective use of `groupby()` will allow you to do lots of really powerful things with your dataset.

### Multi-indexes  <a class="anchor" id="section4_1"></a>

In all of the examples we've seen thus far we've been working with DataFrame or Series objects with a single-label index. `groupby()` is slightly different in the fact that, depending on the operation we run, it will sometimes result in what is called a multi-index.

A multi-index differs from a regular index in that it has multiple levels. For example:


In [ ]:
countries_reviewed = reviews.groupby(["country", "province"]).description.agg([len])
countries_reviewed

In [ ]:
mi = countries_reviewed.index
type(mi)

Multi-indices have several methods for dealing with their tiered structure which are absent for single-level indices. They also require two levels of labels to retrieve a value. Dealing with multi-index output is a common "gotcha" for users new to pandas.

The use cases for a multi-index are detailed alongside instructions on using them in the MultiIndex / Advanced Selection section of the pandas documentation.

However, in general the multi-index method you will use most often is the one for converting back to a regular index, the `reset_index()` method:


In [ ]:
countries_reviewed.reset_index()


### Sorting  <a class="anchor" id="section4_2"></a>

Looking again at `countries_reviewed` we can see that grouping returns data in index order, not in value order. That is to say, when outputting the result of a `groupby`, the order of the rows is dependent on the values in the index, not in the data.

To get data in the order want it in we can sort it ourselves. The `sort_values()` method is handy for this.


In [ ]:
countries_reviewed = countries_reviewed.reset_index()
countries_reviewed.sort_values(by="len")

`sort_values()` defaults to an ascending sort, where the lowest values go first. However, most of the time we want a descending sort, where the higher numbers go first. That goes thusly:

In [ ]:
countries_reviewed.sort_values(by="len", ascending=False)

To sort by index values, use the companion method sort_index(). This method has the same arguments and default order:

In [ ]:
countries_reviewed.sort_index()

Finally, know that you can sort by more than one column at a time

## Additional tools  <a class="anchor" id="chapter5"></a>


### Summary functions  <a class="anchor" id="section5_1"></a>

Pandas provides many simple "summary functions" (not an official name) which restructure the data in some useful way. For example, consider the `describe()` method:


In [ ]:
reviews.points.describe()

This method generates a high-level summary of the attributes of the given column. It is type-aware, meaning that its output changes based on the data type of the input. The output above only makes sense for numerical data; for string data here's what we get:

In [ ]:
reviews.taster_name.describe()

If you want to get some particular simple summary statistic about a column in a DataFrame or a Series, there is usually a helpful pandas function that makes it happen.

For example, to see the mean of the points allotted (e.g. how well an averagely rated wine does), we can use the `mean()` function:


In [ ]:
reviews.points.mean()

To see a list of unique values we can use the `unique()` function:

In [ ]:
reviews.taster_name.unique()

To see a list of unique values and how often they occur in the dataset, we can use the `value_counts()` method:


In [ ]:
reviews.taster_name.value_counts()


### Maps (Advanced skill)  <a class="anchor" id="section5_2"></a>



A **map** is a term, borrowed from mathematics, for a function that takes one set of values and "maps" them to another set of values. In data science we often have a need for creating new representations from existing data, or for transforming data from the format it is in now to the format that we want it to be in later. Maps are what handle this work, making them extremely important for getting your work done!

There are two mapping methods that you will use often.

`map()` is the first, and slightly simpler one. For example, suppose that we wanted to remean the scores the wines received to 0. We can do this as follows:


In [ ]:
review_points_mean = reviews.points.mean()
reviews.points.map(lambda p: p - review_points_mean)

The function you pass to `map()` should expect a single value from the Series (a point value, in the above example), and return a transformed version of that value. `map()` returns a new Series where all the values have been transformed by your function.

`apply()` is the equivalent method if we want to transform a whole DataFrame by calling a custom method on each row.


In [ ]:
def remean_points(row):
    row.points = row.points - review_points_mean
    return row


reviews.apply(remean_points, axis="columns")


If we had called `reviews.apply()` with `axis='index'`, then instead of passing a function to transform each row, we would need to give a function to transform each column.

Note that `map()` and `apply()` return new, transformed Series and DataFrames, respectively. They don't modify the original data they're called on. If we look at the first row of `reviews`, we can see that it still has its original `points` value.


In [ ]:
reviews.head(1)

Pandas provides many common mapping operations as built-ins. For example, here's a faster way of remeaning our points column:

In [ ]:
review_points_mean = reviews.points.mean()
reviews.points - review_points_mean

In this code we are performing an operation between a lot of values on the left-hand side (everything in the Series) and a single value on the right-hand side (the mean value). Pandas looks at this expression and figures out that we must mean to subtract that mean value from every value in the dataset.

Pandas will also understand what to do if we perform these operations between Series of equal length. For example, an easy way of combining country and region information in the dataset would be to do the following:


In [ ]:
reviews.country + " - " + reviews.region_1

These operators are faster than `map()` or `apply()` because they uses speed ups built into pandas. All of the standard Python operators (`>`, `<`, `==`, and so on) work in this manner.

However, they are not as flexible as `map()` or `apply()`, which can do more advanced things, like applying conditional logic, which cannot be done with addition and subtraction alone.

### Dtypes  <a class="anchor" id="section5_3"></a>

The data type for a column in a DataFrame or a Series is known as the **dtype**.

You can use the `dtype` property to grab the type of a specific column. For instance, we can get the dtype of the `price` column in the `reviews` DataFrame:

In [ ]:
reviews.price.dtype

Alternatively, the dtypes property returns the dtype of every column in the DataFrame:

In [ ]:
reviews.dtypes

Data types tell us something about how pandas is storing the data internally. `float64` means that it's using a 64-bit floating point number; `int64` means a similarly sized integer instead, and so on.

One peculiarity to keep in mind (and on display very clearly here) is that columns consisting entirely of strings do not get their own type; they are instead given the `object` type.

It's possible to convert a column of one type into another wherever such a conversion makes sense by using the `astype()` function. For example, we may transform the `points` column from its existing `int64` data type into a `float64` data type:


In [ ]:
reviews.points.astype("float64")

A DataFrame or Series index has its own `dtype`, too:

In [ ]:
reviews.index.dtype

Pandas also supports more exotic data types, such as categorical data and timeseries data. Because these data types are more rarely used, we will omit them until a much later section of this tutorial.


### Missing data  <a class="anchor" id="section5_4"></a>

Entries missing values are given the value NaN, short for "Not a Number". For technical reasons these `NaN` values are always of the `float64` dtype.

Pandas provides some methods specific to missing data. To select `NaN` entries you can use `pd.isnull()` (or its companion `pd.notnull()`). This is meant to be used thusly:


In [ ]:
reviews[pd.isnull(reviews.country)]

Replacing missing values is a common operation. Pandas provides a really handy method for this problem: `fillna()`. `fillna()` provides a few different strategies for mitigating such data. For example, we can simply replace each `NaN` with an `"Unknown"`:

In [ ]:
reviews.region_2.fillna("Unknown")


Or we could fill each missing value with the first non-null value that appears sometime after the given record in the database. This is known as the backfill strategy.





Alternatively, we may have a non-null value that we would like to replace. For example, suppose that since this dataset was published, reviewer Kerin O'Keefe has changed her Twitter handle from @kerinokeefe to @kerino. One way to reflect this in the dataset is using the `replace()` method:


In [ ]:
reviews.taster_twitter_handle.replace("@kerinokeefe", "@kerino")

The `replace()` method is worth mentioning here because it's handy for replacing missing data which is given some kind of sentinel value in the dataset: things like `"Unknown"`, `"Undisclosed"`, `"Invalid"`, and so on.

### Renaming  <a class="anchor" id="section5_5"></a>

The first function we'll introduce here is `rename()`, which lets you change index names and/or column names. For example, to change the `points` column in our dataset to `score`, we would do:

In [ ]:
reviews.rename(columns={"points": "score"})

`rename()` lets you rename index or column values by specifying a `index` or `column` keyword parameter, respectively. It supports a variety of input formats, but usually a Python dictionary is the most convenient. Here is an example using it to rename some elements of the index.

In [ ]:
reviews.rename(index={0: "firstEntry", 1: "secondEntry"})

You'll probably rename columns very often, but rename index values very rarely. For that, `set_index()` is usually more convenient.

Both the row index and the column index can have their own `name` attribute. The complimentary `rename_axis()` method may be used to change these names. For example:


In [ ]:
reviews.rename_axis("wines", axis="rows").rename_axis("fields", axis="columns")


### Combining  <a class="anchor" id="section5_6"></a>

When performing operations on a dataset, we will sometimes need to combine different DataFrames and/or Series in non-trivial ways. Pandas has three core methods for doing this. In order of increasing complexity, these are `concat()`, `join()`, and `merge()`. Most of what `merge()` can do can also be done more simply with `join()`, so we will omit it and focus on the first two functions here.

The simplest combining method is `concat()`. Given a list of elements, this function will smush those elements together along an axis.



In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)
df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)
df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)

In [ ]:
pd.concat([df1, df2, df3])

In [ ]:
pd.concat([df1, df2, df3], axis=1)

In [ ]:
merge_1 = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

merge_2 = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

In [ ]:
pd.merge(merge_1, merge_2, how="inner", on="key")

In [ ]:
join_1 = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)

join_2 = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)

In [ ]:
join_1.join(join_2)